# Other Graph types

The JuliaGraphs ecosystem contains other graph types appart from `SimpleGraph` and `SimpleDiGraph`. In this notebook we are going to look at some of them.

## Setup

In [ ]:
using LightGraphs

# We will mainly looks at the packages
using SimpleWeightedGraphs
using MetaGraphs

# Some helper packages
using BenchmarkTools
using SNAPDatasets
using GraphPlot: gplot, gplothtml
using Random
using Colors: @colorant_str

Execute the next cell to create helper funcion `viz` for displaying graphs.
No need to understand how it works for now.

**!!!** People who use the terminal instead of Jupyter might want to change `gplot` to `gplothtml` though

In [ ]:
function viz(
        g::LightGraphs.AbstractSimpleGraph{T};
        vertex_labels = 1:nv(g),
        color_vertices = Vector{T}(),
        color_edges = Vector{Edge{T}}(),
        edge_labels = String[],
        weights = nothing
    ) where {T}
 
    order_edge(e::Edge) = Edge(minmax(src(e), dst(e)))
    
    color_vertices = Set(color_vertices)
    color_edges = is_directed(g) ? Set(color_edges) : Set(order_edge.(color_edges))
    
    if weights != nothing && isempty(edge_labels)
        edge_labels = map(e -> string(weights[dst(e), src(e)]), edges(g))
    end
    
    vertex_colors = map(v -> v ∈ color_vertices ? colorant"lightblue" : colorant"lightgrey", vertices(g))
    edge_colors = map(e -> e ∈ color_edges ? colorant"red" : colorant"lightgrey", edges(g))
    
    Random.seed!(5)
    gplot(g,
        nodelabel    = vertex_labels,
        edgestrokec  = edge_colors,
        edgelabel    = edge_labels,
        nodefillc    = vertex_colors,
        nodestrokec  = colorant"darkgrey",
        nodestrokelw = 1,
        NODELABELSIZE = 5,
        EDGELABELSIZE = 5
    )
end

viz(gw::AbstractSimpleWeightedGraph; kwargs...) =
    viz(is_directed(gw) ? SimpleDiGraph(gw) : SimpleGraph(gw); weights=adjacency_matrix(gw), kwargs...)


function viz(gm::MetaGraph; kwargs...)
   
    vertex_labels = map(vertices(gm)) do v
        ps = props(gm, v)
        tup = NamedTuple{Tuple(keys(ps))}(Tuple(values(ps)))
        isempty(tup) ? "$v" : "$v: $tup"
    end
    
    edge_labels = map(edges(gm)) do e
        ps = props(gm, src(e), dst(e))
        tup = NamedTuple{Tuple(keys(ps))}(Tuple(values(ps)))
        isempty(tup) ? "" : "$tup"
    end
    
    return viz(is_directed(gm) ? SimpleDiGraph(gm) : SimpleGraph(gm);
        vertex_labels = vertex_labels, edge_labels = edge_labels, kwargs...)
end
;

## Motivation

In [ ]:
g = smallgraph(:house)

In [ ]:
edges(g) |> collect

In [ ]:
viz(g)

### The minimum spanning tree (mst)

Find a minimum connected subgraph of the graph that includes each vertex

In [ ]:
prim_mst(g) 

In [ ]:
viz(g, color_edges = prim_mst(g))

### Weighted minimum spannning tree

Add a weight to each edge and select the spanning tree with minimum (sum over all weights) weight

In [ ]:
weights = adjacency_matrix(g) |> Matrix
weight_matrix[1, 3] = 100; weight_matrix[3, 1] = 100;
weight_matrix

In [ ]:
viz(g, weights=weight_matrix)

In [ ]:
prim_mst(g, weight_matrix)

In [ ]:
viz(g, color_edges=prim_mst(g, weight_matrix),  weights=weight_matrix)

## Exercise 1: Change `weights` so that the minimum spanning tree uses the edges (1,3) (2,4), (3,4) and (4, 5) 

Solution below

.

.

.

.

.

.

.

.

.

.



.

.

.

.

.

.


In [ ]:
weight_matrix = adjacency_matrix(g)
weight_matrix[1, 2] = 100; weight_matrix[2, 1] = 100;
weight_matrix[3, 5] = 100; ws[5, 3] = 100;

viz(g, color_edges=prim_mst(g, weight_matrix),  weights=weight_matrix)


### Can we store the edge weights (distances) in the graph?

<br>
<br>
<br>



## SimpleWeightedGraphs.jl

- Two types: `SimpleWeightedGraph` and `SimpleWeightedDiGraph`
- Wrapper around a `SparseMatrixCSC`
- Works with most `LightGraphs` functions that take a weight (distance) matrix

In [ ]:
using SimpleWeightedGraphs

In [ ]:
# Create from SimpleGraph

gw = SimpleWeightedGraph(smallgraph(:house))

In [ ]:
viz(gw)

In [ ]:
# Create from a matrix
A = Float64[
    0  4  2
    4  0  1
    2  1  0
]

In [ ]:
gw2 = SimpleWeightedGraph(A)
viz(gw2)

In [ ]:
viz(gw2, color_edges = prim_mst(gw2))

In [ ]:
add_edge!(gw2, 2, 3, 100)

viz(gw2)

In [ ]:
add_edge!(gw2, 2, 3, 0) # ignored because zeros represent non-existing edges

viz(gw2)

In [ ]:
rem_edge!(gw2, 2, 3)

viz(gw2)

In [ ]:
adjacency_matrix(gw2) |> Matrix

### Exercise 2: Create a directed weighted star graph
- Create a matrix that represents the weighted adjacency matrix of the following graph
- Ensure that edge `(1, n)` has weight `n`
- Convert the graph to `SimpleWeightedDiGraph`

In [ ]:
viz(star_digraph(4)) # directed star graph without weights

Solution below

.

.

.

.

.

.

.

.

.

.


.

.

.

.

.

.


In [ ]:
# Note source of edge is column number not row number

A = [
    0 0 0 0
    2 0 0 0
    3 0 0 0
    4 0 0 0
]

gwd = SimpleWeightedDiGraph(A)

viz(gwd)

<br>
<br>
<br>

### Is there a way to have more than one property on an edge?

## MetaGraphs.jl

- Two types: `MetaGraph` and `MetaDiGraph`
- Wrapper around `SimpleGraph` and `SimpleDiGraph`


In [ ]:
using MetaGraphs

In [ ]:
gm = MetaGraph(smallgraph(:bull))
viz(gm)

In [ ]:
# setting multipe edge properties

set_prop!(gm, 1, 2, :a, 10)
set_prop!(gm, 1, 2, :c, 0.2)

set_prop!(gm, 2, 3, :a, 5)
set_prop!(gm, 2, 3, :b, "hello")
set_prop!(gm, 2, 3, :c,  0.1)

set_prop!(gm, 2, 4, :c, 7.0)

viz(gm)

In [ ]:
# retrieve single edge prop

get_prop(gm, 2, 3, :a) |> println
get_prop(gm, 2, 3, :b) |> println


# retrieve all edge props
props(gm, 2, 3) |> println

In [ ]:
# setting a vertex property

set_prop!(gm, 5, :name, "vertex 5")

viz(gm)

In [ ]:
# setting a graph property
set_prop!(gm, :graph_name, "bull graph")

get_prop(gm, :graph_name)

In [ ]:
# Running minimum spanning tree

viz(gm, color_edges = prim_mst(gm))

In [ ]:
# Which edge property is used for the spanning tree?


LightGraphs.weights(gm) |> Matrix

In [ ]:
weightfield(gm) # edge property that is used in the algorithm

In [ ]:
# We can change the used edge property:

weightfield!(gm, :a)

viz(gm, color_edges = prim_mst(gm))

### Exercise 3: Change the weightfield to :c
- Think how the minimum spanning tree would change if you change the weighfield to :c
- Then change it and plot the new spanning tree

Solution below
.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

In [ ]:
weightfield!(gm, :c)
viz(gm, color_edges = prim_mst(gm))

## When to use which graph type?

### Benchmark:

In [ ]:
using SNAPDatasets, BenchmarkTools

g = loadsnap(:email_enron)

In [ ]:
gw = SimpleWeightedGraph(g)
gm = MetaGraph(g)

In [ ]:
for e in edges(g)
    add_edge!(gw, src(e), dst(e), 2.0)
    set_prop!(gm, src(e), dst(e), :weight, 2.0)
end
A = adjacency_matrix(gw);

In [ ]:
@btime prim_mst(g, A);

In [ ]:
@btime prim_mst(gw);

In [ ]:
@btime prim_mst(gm);

### Conclusion

- MetaGraph much slower.

#### SimpleGraphs
- No edge property is needed
- Lots of structural changes to graph

#### SimpleWeightedGraphs
- Single numerical edge property
- No vertex/graph properties
- Not many structural changes to graph

#### MetaGraphs
- Different or non-numerical edge properties
- Vertex/graph properties
- Lots of structural changes to graph

## Other graph types

- [StaticGraphs.jl](https://github.com/JuliaGraphs/StaticGraphs.jl)
    - SimpleGraphs with more cache optimal layout
    - Immutable

- [SpecialGraphs.jl](https://github.com/JuliaGraphs/SpecialGraphs.jl)
    - Parametrized graphs where we only have to store the parameter
    - Example: complete graph `K_n` - it is sufficient to store `n`
    - Immutable
    - Can specialize certain algorithms


- [LightGraphsGraphBlas.jl](https://github.com/abhinavmehndiratta/LightGraphsGraphBLAS.jl])
    - Created by Abhinav Mehndiratta for last summers Gsoc
    - Wraps graphs that implement the GraphBLAS Api into LightGraphs types

## Summary and outlook

- SimpleWeightedGraphs for sinle nummeric properties
- MetaGraphs for multiple or non-numeric properties

=> Huge possibilty for other graph type